### Trading Testing

Testing delta probability for weeklies and monthlies
    - Does delta match probability of occurrence
    - Check Kelly criterion for long straddles/delta neutral back ratios
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage
        moves and use the normal pdf as the probability weightings
    
Check earnings returns post announcement factors
    - Factors:
        - Number of times beaten earnings (Dummy Variable)
        - Consecutive earnings beats
        - Consecutive earnings upsets
        - 3 Month Trend before earnings
        - YTD Trend
        - Momentum of monthly returns (20 day, 60 day)


In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *

from scipy.stats import norm as norm
from yahoo_query import *
# '''
# functions list:
    
#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]
    
#     optionslam_scrape(str[ticker]) --> dict[earnings]
    
#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]
    
#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]
    
#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]
    
#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]
    
#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]
    
#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]
    
#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]
    
#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]
    
#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()
    
#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]
    
#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]
    
#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

In [2]:
# Pulling Indices Data

spx = pd.read_csv('spx.csv', index_col = 0)[['Open','Close']]
spx.columns = ['spx_{0}'.format(x) for x in spx.columns]

vix = pd.read_csv('vix.csv', index_col = 0)[['Open','Close']]
vix.columns = ['vix_{0}'.format(x) for x in vix.columns]

vvix = pd.read_csv('vvix.csv', index_col = 0)[['Close']]
vvix.columns = ['vvix']

yields = pd.read_csv('https://www.quandl.com/api/v3/datasets/USTREASURY/YIELD.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)
df = pd.concat([spx,yields[yields.columns[1:]]], axis = 1).dropna()
df.index = pd.to_datetime(df.index)
df[df.columns] = df[df.columns].apply(pd.to_numeric, errors='coerce')

In [3]:
# Creating full calendar

nyse_dates = nyse.schedule(start_date=df.index[0].date().strftime('%Y-%m-%d'), end_date=df.index[-1].date().strftime('%Y-%m-%d'))
date_idx = [x.date() for x in mcal.date_range(nyse_dates, frequency='1D')]

date_df = pd.DataFrame(columns = ['Trading_Day'],
                       index = range(len(date_idx)))
date_df.index = date_idx
date_df.index = pd.to_datetime(date_df.index)
date_df['Trading_Day'] = True

start_date = dt.datetime(1993,12,1).date()

full_cal = []
curr_date = start_date
while curr_date != dt.datetime.today().date():
    full_cal.append(curr_date)
    curr_date = curr_date + dt.timedelta(days = 1)
    
calendar_df = pd.DataFrame(columns = ['Weekday'],
                           index = full_cal)
calendar_df.index = pd.to_datetime(calendar_df.index)
calendar_df['Weekday'] = calendar_df.index.weekday
date_df = calendar_df.join(date_df).fillna(False)

In [4]:
spx_check = date_df.join(df)
spx_check = spx_check[spx_check['Weekday'].isin([0,1,2,3,4])]
spx_check['Week'] = spx_check.index.week
spx_check['Month'] = spx_check.index.month
spx_check['Year'] = spx_check.index.year
spx_check['Month_Expiry'] = False
spx_check['New_Month'] = False
spx_check['Week_Start'] = False
spx_check['Week_End'] = False

# Marking Monthly Expirations for SPX and Following Trading Day
for idx, row in spx_check[['Month','Year']].drop_duplicates().iterrows():
    curr_eigth_day = dt.date(row['Year'],row['Month'],7)
    curr_third_fri = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],3).weekday()) + dt.timedelta(14)
    curr_third_thurs = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],4).weekday()) + dt.timedelta(14)
    curr_third_wed = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],5).weekday()) + dt.timedelta(14)
    
    # Marking monthly expiry days: Third Friday of every month, otherwise third thursday
    if spx_check.loc[curr_third_fri,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_fri,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_thurs,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_thurs,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_wed,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_wed,'Month_Expiry'] = True
        
# Marking Monthly Starting
for idx, row in spx_check[spx_check['Month_Expiry'] == True].iterrows():
    following_week = spx_check[spx_check.index > idx].head()
    for idx_2, row_2 in following_week.iterrows():
        if row_2['Trading_Day']:
            spx_check.loc[idx_2, 'New_Month'] = True
            break
            
# Setting Weekly Tags

for idx, row in spx_check[['Week','Year']].drop_duplicates().iterrows():
    current_week = spx_check[(spx_check['Week'] == row['Week']) &
                             (spx_check['Year'] == row['Year']) &
                             (spx_check['Trading_Day'] == True)]
    week_start = current_week.head(1)
    week_end = current_week.tail(1)
    
    try:
        spx_check.loc[week_start.index[0].date(), 'Week_Start'] = True
        spx_check.loc[week_end.index[0].date(), 'Week_End'] = True
    except:
        continue

spx_check['Monthlies'] = spx_check['Month_Expiry'] | spx_check['New_Month']
spx_monthly_check = spx_check[spx_check['Monthlies'] == True]

spx_check['Weeklies'] = spx_check['Week_Start'] | spx_check['Week_End']
spx_weekly_check = spx_check[(spx_check['Weeklies'] == True) &
                             (spx_check['Year'] > 1993) &
                             (spx_check['Year'] < 2018)]

# Filling in missing values
spx_monthly_check.spx_Open = spx_monthly_check.spx_Open.fillna((spx_monthly_check.spx_Open.shift() + 
                                                                spx_monthly_check.spx_Open.shift(-1))/2)
spx_monthly_check.spx_Close = spx_monthly_check.spx_Close.fillna((spx_monthly_check.spx_Close.shift() + 
                                                                  spx_monthly_check.spx_Close.shift(-1))/2)

spx_weekly_check.spx_Open = spx_weekly_check.spx_Open.fillna((spx_weekly_check.spx_Open.shift() + 
                                                              spx_weekly_check.spx_Open.shift(-1))/2)
spx_weekly_check.spx_Close = spx_weekly_check.spx_Close.fillna((spx_weekly_check.spx_Close.shift() + 
                                                                spx_weekly_check.spx_Close.shift(-1))/2)

C:\Users\Fang\Anaconda3\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
spx_weekly_check = spx_weekly_check[['spx_Open','spx_Close','Week_Start','Week_End','Year'] + list(yields.columns[1:])]
spx_weekly_check['Week_Return'] = spx_weekly_check.spx_Close/spx_weekly_check.spx_Open.shift(1) - 1
spx_weekly_check = spx_weekly_check[spx_weekly_check['Week_End'] == True]
spx_weekly_check['Up_Moves'] = (spx_weekly_check['Week_Return'] > 0.005)*1
spx_weekly_check['Down_Moves'] = (spx_weekly_check['Week_Return'] < -0.005)*1
spx_weekly_check['No_Moves'] = 1 - spx_weekly_check['Up_Moves'] - spx_weekly_check['Down_Moves']

weekly_counts = {'Ups':[],'Downs':[],'Flats':[]}
weekly_tally = spx_weekly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_weekly_check.Year.drop_duplicates():
    weekly_counts['Ups'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Up_Moves'])
    weekly_counts['Downs'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Down_Moves'])
    weekly_counts['Flats'].append(weekly_tally[weekly_tally.index.year == yr].sum()['No_Moves'])
    
weekly_df = pd.DataFrame(weekly_counts, index = spx_weekly_check.Year.drop_duplicates())
weekly_df = weekly_df.drop([2002,2003,2004,2005,2006])
weekly_df_pct = pd.concat([(weekly_df['Ups']/weekly_df.sum(axis = 1)),
                           (weekly_df['Downs']/weekly_df.sum(axis = 1)),
                           (weekly_df['Flats']/weekly_df.sum(axis = 1))],axis = 1)
weekly_df_pct.columns = ['UpMove','DownMove','NoMove']
weekly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1994,0.365385,0.288462,0.346154
1995,0.576923,0.173077,0.250000
1996,0.519231,0.326923,0.153846
1997,0.461538,0.365385,0.173077
1998,0.547170,0.320755,0.132075
1999,0.519231,0.365385,0.115385
2000,0.423077,0.538462,0.038462
2001,0.442308,0.442308,0.115385
2007,0.480769,0.346154,0.173077


In [6]:
spx_monthly_check = spx_monthly_check[['spx_Open','spx_Close','New_Month','Month_Expiry','Year'] + list(yields.columns[1:])]
spx_monthly_check['Month_Return'] = spx_monthly_check.spx_Close/spx_monthly_check.spx_Open.shift(1) - 1
spx_monthly_check = spx_monthly_check[spx_monthly_check['Month_Expiry'] == True]
spx_monthly_check['Up_Moves'] = (spx_monthly_check['Month_Return'] > 0.01)*1
spx_monthly_check['Down_Moves'] = (spx_monthly_check['Month_Return'] < -0.01)*1
spx_monthly_check['No_Moves'] = 1 - spx_monthly_check['Up_Moves'] - spx_monthly_check['Down_Moves']

monthly_counts = {'Ups':[],'Downs':[],'Flats':[]}
monthly_tally = spx_monthly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_monthly_check.Year.drop_duplicates():
    monthly_counts['Ups'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Up_Moves'])
    monthly_counts['Downs'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Down_Moves'])
    monthly_counts['Flats'].append(monthly_tally[monthly_tally.index.year == yr].sum()['No_Moves'])
    
monthly_df = pd.DataFrame(monthly_counts, index = spx_monthly_check.Year.drop_duplicates())
monthly_df = monthly_df.drop([2002,2003,2004,2005,2006])
monthly_df_pct = pd.concat([(monthly_df['Ups']/monthly_df.sum(axis = 1)),
                           (monthly_df['Downs']/monthly_df.sum(axis = 1)),
                           (monthly_df['Flats']/monthly_df.sum(axis = 1))],axis = 1)
monthly_df_pct.columns = ['UpMove','DownMove','NoMove']
monthly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1993,0.000000,0.000000,1.000000
1994,0.333333,0.250000,0.416667
1995,0.916667,0.000000,0.083333
1996,0.583333,0.166667,0.250000
1997,0.583333,0.333333,0.083333
1998,0.666667,0.250000,0.083333
1999,0.416667,0.166667,0.416667
2000,0.333333,0.583333,0.083333
2001,0.416667,0.416667,0.166667


In [7]:
monthly_df

,Downs,Flats,Ups
Year,,,
1993,0,1,0
1994,3,5,4
1995,0,1,11
1996,2,3,7
1997,4,1,7
1998,3,1,8
1999,2,5,5
2000,7,1,4
2001,5,2,5


In [150]:
contracts = spx_options.loc[[49,17]]
holding_days = 2
positions = [1, 1]
day_format = 'trading'
prem_price_use = 'Mid'

def ev_calc(contracts, positions, holding_days, price_stds = 5, day_format = 'trading', prem_price = 'Mid'):
    vix = curr_stock_data('^VIX')[-1]/100

    if day_format != 'trading':
        year = 365
    else:
        year = 252

    def kelly(odds, p, q):
        return (odds*p - q)/odds

    period_std = vix/np.sqrt(year/holding_days)

    returns = np.linspace(-price_stds*period_std,price_stds*period_std, 300)
    returns = pd.DataFrame({'Returns': returns,
                            'CDF': norm.cdf(returns, 0, period_std)}, index = range(len(returns)))

    returns['probs'] = round(returns['CDF'] - returns['CDF'].shift(1), 6).fillna(0)
    sims = []
    for idx, row in returns.iterrows():
        sims.append(position_sim(contracts, positions, 0, row.Returns, 0, holding_days, prem_price_use = prem_price))

    sim_df = pd.concat(sims,axis = 0)
    sim_df.index = range(len(returns))
    sim_df = sim_df.join(returns)[['PnL','probs','Percent Return']]

    sim_df['EV'] = sim_df.probs*sim_df.PnL
    expected_val = sim_df.EV.sum()
    expected_gain = sim_df[sim_df['EV'] >= 0].EV.sum()
    expected_loss = sim_df[sim_df['EV'] < 0].EV.sum()
    gain_prob = sim_df[sim_df['EV'] >= 0].probs.sum()
    loss_prob = sim_df[sim_df['EV'] < 0].probs.sum()

    out_df = pd.DataFrame({'EV': expected_val,
                           'Expected Win': expected_gain,
                           'Prob Win': gain_prob,
                           'Expected Loss': expected_loss,
                           'Prob Loss': loss_prob,
                           'Net Cost': (contracts.Mid*contracts.Pos).sum()*100,
                           'Kelly Fraction': kelly(expected_gain/((contracts.Mid*contracts.Pos).sum()*100), gain_prob, loss_prob)},
                          index = [0])
    
    return out_df

In [2]:
stock_list = pd.read_csv('optionablestocks.csv')

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')

    earnings_moves = past_earnings(ticker).sort_index()
    earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
                                    (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()

    earnings_df = pd.concat([earnings_info.reset_index(), 
                             earnings_moves.reset_index()], axis = 1)
    earnings_df.columns = ['quarter' if col == 'index' else col for col in earnings_df.columns.tolist()]
    earnings_df['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')
    
    return (earnings_df, keyMetrics)

def download_yahoo_data(ticker_list, retries = 5):

    earnings_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print("{0:.2f}% Completed".format(item_counter/total_length*100))
        print("{} total failures".format(len(failed_list)))
        
        earnings_df = pd.concat(earnings_lst, axis = 0)
        earnings_df = earnings_df.reset_index()[earnings_df.columns]
        keyStats_df = pd.concat(keyStats_lst, axis = 0)
        
        return earnings_df, keyStats_df, failed_list

# earnings_df, keyMetrics = pull_data('AMD')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [2]:
#earnings_testing = earnings_df.dropna(subset = ['1Year','1month','3month','6month']).reset_index()[earnings_df.columns]



earnings = pd.read_csv('earnHist_v2.csv')

FileNotFoundError: File b'earnHist_v2.csv' does not exist

In [47]:
earnings_testing

,1Year,1month,3month,6month,PostEarningsReturn,Underlying,accountsPayable,capitalExpenditures,capitalSurplus,cash,...,totalCashFromOperatingActivities,totalCashflowsFromInvestingActivities,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,totalStockholderEquity,treasuryStock
0,0.225846,-0.029897,0.044871,0.178408,0.045432,A,305000000,-58000000,5300000000,2678000000,...,288000000,-60000000,4169000000,1263000000,3591000000,919000000,-44000000,1189000000,4831000000,-346000000
1,0.553286,0.043868,0.111182,0.192636,-0.000715,A,292000000,-60000000,5320000000,2887000000,...,215000000,-67000000,4397000000,1361000000,4172000000,972000000,-6000000,1211000000,4522000000,-272000000
2,0.383211,-0.032750,0.038284,0.173796,0.032744,A,271000000,-48000000,5332000000,3011000000,...,303000000,-50000000,4525000000,1365000000,4167000000,986000000,7000000,1206000000,4613000000,-310000000
3,0.232372,0.015256,0.011399,0.025789,-0.101979,A,273000000,-33000000,5312000000,2131000000,...,197000000,-473000000,3667000000,1014000000,3781000000,978000000,17000000,1203000000,4564000000,-335000000
4,0.362637,-0.094513,0.128152,0.316748,-0.027802,AA,1813000000,-74000000,9633000000,1196000000,...,55000000,-74000000,4149000000,2976000000,9579000000,2671000000,-7000000,3090000000,5368000000,-4530000000
5,0.746017,0.337479,0.181385,0.564370,-0.072584,AA,1752000000,-95000000,9650000000,1089000000,...,-430000000,-100000000,4281000000,3013000000,9410000000,2936000000,-233000000,3579000000,5024000000,-4967000000
6,0.870277,0.278244,0.042288,0.231343,0.013876,AA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.381215,0.015228,0.136880,0.008268,0.009752,AAL,1638000000,-1369000000,5918000000,340000000,...,369000000,184000000,10002000000,15307000000,48453000000,9534000000,-340000000,10878000000,3948000000,-5127000000
8,0.285462,0.089007,0.020400,0.160073,-0.048388,AAL,1688000000,-1408000000,5714000000,295000000,...,437000000,-557000000,9146000000,14964000000,47470000000,9631000000,-545000000,10601000000,3926000000,-5154000000
9,0.152503,0.041833,0.074735,0.082592,-0.032273,AAL,1953000000,-779000000,5279000000,297000000,...,1799000000,-956000000,9673000000,17008000000,54298000000,9775000000,-353000000,10401000000,-1018000000,-5172000000
